In [ ]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from qwenimage.experiment import ExperimentConfig
from qwenimage.experiments.experiments_qwen import ExperimentRegistry


# experiment_names = ExperimentRegistry.keys()
experiment_names = [

    "qwen_base",
    "qwen_fa3",
    "qwen_aot",
    "qwen_fuse_aot",
    "qwen_fa3_aot",

    "qwen_fbcache_052",
    "qwen_fbcache_053",
    "qwen_fbcache_054",
    "qwen_fbcache_055",

    "qwen_fa3_aot_fp8",
    "qwen_fa3_aot_fp8_fuse",

    "qwen_base_3step",
    "qwen_base_2step",
    "qwen_fa3_aot_int8_fuse_3step",
    "qwen_fa3_aot_int8_fuse_2step",

    "qwen_lightning_lora",
    "qwen_lightning_lora_2step",
    "qwen_lightning_lora_3step",
    "qwen_lightning_fa3_aot_int8_fuse_3step",
    "qwen_lightning_fa3_aot_int8_fuse_2step",
    "qwen_lightning_fa3_aot_fp8_fuse",

]

report_dir = ExperimentConfig().report_dir

all_results = []
for name in experiment_names:
    csv_path = report_dir / f"{name}.csv"
    
    df = pd.read_csv(csv_path, index_col=0)
    df['experiment'] = name
    all_results.append(df)
    print(f"Loaded results for {name}: {len(df)} rows")



In [ ]:
all_results[0]

In [ ]:
combined_df = pd.concat(all_results, ignore_index=True)
print(f"{combined_df.shape=}")
print(f"{combined_df.columns.tolist()=}")
combined_df.head(5)

In [ ]:
# profile_targets = ["loop", "run_once"]
profile_targets = {
    "loop": "equal",
    "run_once": "contain"
}

for target, match_strategy in profile_targets.items():
    if match_strategy == "equal":
        plot_data = combined_df[combined_df['name'] == target].copy()
    elif match_strategy == "contain":
        plot_data = combined_df[combined_df['name'].str.contains(target, case=False, na=False)].copy()
    else:
        raise ValueError()
    plot_data = plot_data.sort_values('mean', ascending=False)

    fig, ax = plt.subplots(figsize=(12, 6))
    x_pos = range(len(plot_data))
    max_time = plot_data['mean'].max()

    bars = ax.bar(x_pos, plot_data['mean'], yerr=plot_data['std'], 
                    capsize=12, alpha=0.7, edgecolor='black')
    ax.set_xlabel('Optimization Type', fontsize=12, fontweight='bold')
    ax.set_ylabel('Time (s)', fontsize=12, fontweight='bold')
    ax.set_title(f'Optimizations Comparison Over: {target}', 
                    fontsize=14, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([row['experiment'] for _, row in plot_data.iterrows()], 
                        rotation=45, ha='right', fontsize=12)
    ax.grid(axis='y', alpha=0.3)


    for i, (idx, row) in enumerate(plot_data.iterrows()):        
        ax.text(i - 0.2, row['mean'] + 0.01, f"{row['mean']:.3f}s", 
                ha='center', va='bottom', fontsize=12)
        
        pct_decrease = ((max_time - row['mean']) / max_time) * 100
        ax.text(i + 0.2, row['mean'] + 0.01, f"(-{pct_decrease:.1f}%)", 
                ha='center', va='bottom', fontsize=12, color='green')
    
    

    plt.tight_layout()

    plot_path = report_dir / f'{target}_performance_comparison.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')

    plt.show()


In [ ]:
stack_targets = ["Preprocessing", "Encode Prompt", "Prep gen", "loop", "pre decode", "vae.decode", "post process", "offload"]


stack_data = combined_df[combined_df['name'].isin(stack_targets)].copy()
pivot_data = stack_data.pivot(index='experiment', columns='name', values='mean')
pivot_data = pivot_data[stack_targets]
pivot_data['total'] = pivot_data.sum(axis=1)
pivot_data = pivot_data.sort_values('total', ascending=False)
pivot_data = pivot_data.drop('total', axis=1)


fig, ax = plt.subplots(figsize=(14, 7))
pivot_data.plot(kind='bar', stacked=True, ax=ax, 
                colormap='viridis', edgecolor='black', capsize=12, alpha=0.7, width=0.8)


ax.set_xlabel('Optimization Type', fontsize=12, fontweight='bold')
ax.set_ylabel('Time (s)', fontsize=12, fontweight='bold')
ax.set_title('Pipeline Time Breakdown by Optimization', 
             fontsize=14, fontweight='bold')
ax.set_xticklabels(pivot_data.index, rotation=45, ha='right', fontsize=12)
ax.legend(title='Pipeline Stage', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
ax.grid(axis='y', alpha=0.3)

max_time = pivot_data.sum(axis=1).max()
for i, (idx, row) in enumerate(pivot_data.iterrows()):
    total = row.sum()
    
    ax.text(i - 0.2, total + 0.01, f'{total:.3f}s', 
            ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    pct_decrease = ((max_time - total) / max_time) * 100
    ax.text(i + 0.2, total + 0.01, f'(-{pct_decrease:.1f}%)', 
            ha='center', va='bottom', fontsize=12, color='green', fontweight='bold')

plt.tight_layout()

plot_path = report_dir / 'stacked_time_breakdown.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
stack_data